In [114]:
import requests
import pandas as pd
import json

In [115]:
import plotly.io as pio
# for web, pour pouvoir exporter les figures. Peut être mis juste avant le code concernant la figure
#pio.renderers.default = "svg"   
# workspaces:
pio.renderers.default = "vscode"

import plotly.graph_objects as go
import plotly.express as px

# 1. Coordinates

In [2]:
ville = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg",
        "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille",
        "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege",
        "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [7]:
end_point = "https://nominatim.openstreetmap.org/"
all_city = pd.DataFrame()
for i in range(len(ville)):
    city = pd.DataFrame(requests.get(end_point + "search", params = {'city' : ville[i], 'state' : 'France', 'format' : 'json'}).json())

    if city.shape[0] == 0: # case of Gorges du Verdon
        city = pd.DataFrame(requests.get(end_point + "search", params = {'street' : ville[i], 'state' : 'France', 'format' : 'json'}).json())

    if city.shape[0] > 1:
        city = city.loc[(city["importance"] == city["importance"].max())]
    if city.shape[0] > 1: # case of Paris only
        city = city = city.loc[(city["osm_id"] == city["osm_id"].min())]
        
    city = city[["place_id", "lat", "lon", "display_name"]]
    city["ville"] = ville[i]
    all_city = pd.concat([all_city, city])

print(all_city.shape)
all_city.head()

(35, 5)


,place_id,lat,lon,display_name,ville
0,156094736,48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",Mont Saint Michel
0,297756747,48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",St Malo
0,297981358,49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",Bayeux
0,298137491,49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",Le Havre
0,297518815,49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",Rouen


In [4]:
# save this csv for scraping
all_city.to_csv('src/coordinates.csv')

# 2. Weather data

In [5]:
API_key = ""
end_point = "https://api.openweathermap.org/data/2.5/"

# need city and list information from json fil from request
# function to format the list of weather data
    # @data must be in the format dict['list'][index_element]
def extractWeather(data):
    data_tr = pd.json_normalize(data)
    for c in data['weather'][0].keys():    
        data_tr.insert(len(data_tr.columns), column=("mainweather." + c), value=data['weather'][0][c])
    return(data_tr)

In [6]:
all_forecast = pd.DataFrame()
for i in ville:
    lat = all_city.loc[(all_city["ville"] == i), "lat"]
    lon = all_city.loc[(all_city["ville"] == i), "lon"]
    ## request
    forecast = requests.get(end_point + "forecast", params = {"lat" : lat, "lon" : lon, "appid" : API_key, "units" : "metric"}).json()

    ## steps on forecast
        ## extracting and formatting weather data
    data_w = pd.DataFrame()
    for l in range(len(forecast['list'])):
        this_date = extractWeather(forecast['list'][l])
        data_w = pd.concat([data_w, this_date])
    data_w = data_w.drop("weather", axis=1)

        ## extracting city information
    city = pd.json_normalize(forecast['city'])
        ## adding city information to weather dataframe
    for c in city.columns:    
        data_w.insert(len(data_w.columns), column=c, value=city[c])

    #display(data_w.head(2))

    ## final concatenation
    all_forecast = pd.concat([all_forecast, data_w])

display(all_forecast.head())
print(all_forecast.shape)

,dt,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,...,mainweather.icon,id,name,country,population,timezone,sunrise,sunset,coord.lat,coord.lon
0,1667304000,10000,0.39,2022-11-01 12:00:00,16.39,15.96,16.39,16.72,1015,1015,...,10d,6435453,Huisnes-sur-Mer,FR,0,3600,1667285522,1667321226,48.636,-1.5115
0,1667314800,10000,1.00,2022-11-01 15:00:00,15.95,15.56,15.06,15.95,1015,1015,...,10d,6435453,Huisnes-sur-Mer,FR,0,3600,1667285522,1667321226,48.636,-1.5115
0,1667325600,10000,1.00,2022-11-01 18:00:00,14.13,13.61,13.00,14.13,1016,1016,...,10n,6435453,Huisnes-sur-Mer,FR,0,3600,1667285522,1667321226,48.636,-1.5115
0,1667336400,10000,0.07,2022-11-01 21:00:00,11.60,10.80,11.60,11.60,1019,1019,...,01n,6435453,Huisnes-sur-Mer,FR,0,3600,1667285522,1667321226,48.636,-1.5115
0,1667347200,10000,0.15,2022-11-02 00:00:00,10.90,10.11,10.90,10.90,1020,1020,...,02n,6435453,Huisnes-sur-Mer,FR,0,3600,1667285522,1667321226,48.636,-1.5115


(1400, 32)


In [7]:
print(len(all_forecast["name"].unique()))

35


In [8]:
# save this csv for scraping
all_forecast.to_csv('src/forecast.csv')

# 3. Creation of selection criteria based on the weather

In [7]:
all_city = pd.read_csv("src/coordinates.csv")
all_forecast = pd.read_csv("src/forecast.csv")

## 3.1 preparation of the dataframe

In [8]:
all_forecast = all_forecast.drop(columns='Unnamed: 0', axis=1)
all_forecast = all_forecast.drop(columns='id', axis=1)
all_forecast.columns

Index(['dt', 'visibility', 'pop', 'dt_txt', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.sea_level',
       'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all',
       'wind.speed', 'wind.deg', 'wind.gust', 'rain.3h', 'sys.pod',
       'mainweather.id', 'mainweather.main', 'mainweather.description',
       'mainweather.icon', 'name', 'country', 'population', 'timezone',
       'sunrise', 'sunset', 'coord.lat', 'coord.lon'],
      dtype='object')

In [9]:
print("\ncity name in all_forecast df:\n", all_forecast["name"].unique())
print("\ncity name in all_city df:\n", all_city["ville"].unique())

# preparation for merging
print("\ncity caracteristics in all_city df:\n", all_city.loc[all_city["ville"] == "Amiens",["ville", "lat", "lon"]])
print("\ncity caracteristics in all_forecast df:\n", all_forecast.loc[all_forecast["name"] == "Amiens",["name", "coord.lat", "coord.lon"]].head(1))

## need to round the coordinate values
all_forecast["lat"] = round(all_forecast["coord.lat"], 2)
all_forecast["lon"] = round(all_forecast["coord.lon"], 2)

all_city["lat"] = round(all_city["lat"], 2)
all_city["lon"] = round(all_city["lon"], 2)
## specifica case for Montauban where the rounded value is different to city df
all_forecast.loc[all_forecast["name"] == "Montauban", ["lon"]] = all_city.loc[all_city["ville"] == "Montauban", ["lon"]].values

# merging
all_forecast = all_forecast.merge(all_city.loc[:,["ville", "lat", "lon"]], on=["lat", "lon"])
all_forecast.head(5)


city name in all_forecast df:
 ['Huisnes-sur-Mer' 'St-Malo' 'Bayeux' 'Le Havre' 'Rouen' 'Palais-Royal'
 'Amiens' 'Lille' 'Strasbourg' 'Saint-Hippolyte' 'Colmar' 'Eguisheim'
 'Besançon' 'Larrey' 'Annecy' 'Grenoble' 'Vieux Lyon' 'Bauduen'
 'Bormes-les-Mimosas' 'Cassis' 'Marseille' 'Aix-en-Provence' 'Avignon'
 'Uzès' 'Nîmes' 'Aigues-Mortes' 'Saintes-Maries-de-la-Mer' 'Collioure'
 'Carcassonne' 'Thomery' 'Toulouse' 'Montauban' 'Biarritz' 'Bayonne'
 'La Rochelle']

city name in all_city df:
 ['Mont Saint Michel' 'St Malo' 'Bayeux' 'Le Havre' 'Rouen' 'Paris'
 'Amiens' 'Lille' 'Strasbourg' 'Chateau du Haut Koenigsbourg' 'Colmar'
 'Eguisheim' 'Besancon' 'Dijon' 'Annecy' 'Grenoble' 'Lyon'
 'Gorges du Verdon' 'Bormes les Mimosas' 'Cassis' 'Marseille'
 'Aix en Provence' 'Avignon' 'Uzes' 'Nimes' 'Aigues Mortes'
 'Saintes Maries de la mer' 'Collioure' 'Carcassonne' 'Ariege' 'Toulouse'
 'Montauban' 'Biarritz' 'Bayonne' 'La Rochelle']

city caracteristics in all_city df:
     ville        lat       

,dt,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,...,country,population,timezone,sunrise,sunset,coord.lat,coord.lon,lat,lon,ville
0,1667304000,10000,0.39,2022-11-01 12:00:00,16.39,15.96,16.39,16.72,1015,1015,...,FR,0,3600,1667285522,1667321226,48.636,-1.5115,48.64,-1.51,Mont Saint Michel
1,1667314800,10000,1.00,2022-11-01 15:00:00,15.95,15.56,15.06,15.95,1015,1015,...,FR,0,3600,1667285522,1667321226,48.636,-1.5115,48.64,-1.51,Mont Saint Michel
2,1667325600,10000,1.00,2022-11-01 18:00:00,14.13,13.61,13.00,14.13,1016,1016,...,FR,0,3600,1667285522,1667321226,48.636,-1.5115,48.64,-1.51,Mont Saint Michel
3,1667336400,10000,0.07,2022-11-01 21:00:00,11.60,10.80,11.60,11.60,1019,1019,...,FR,0,3600,1667285522,1667321226,48.636,-1.5115,48.64,-1.51,Mont Saint Michel
4,1667347200,10000,0.15,2022-11-02 00:00:00,10.90,10.11,10.90,10.90,1020,1020,...,FR,0,3600,1667285522,1667321226,48.636,-1.5115,48.64,-1.51,Mont Saint Michel


In [10]:
from datetime import datetime

# date format
def create_features(df):
    df = df.copy()

    df['date']      = pd.to_datetime(df['dt_txt'], utc=True)
    df['dayofweek'] = df['date'].dt.dayofweek
    
    df['dt_sunrise'] = df['sunrise'].apply(lambda x : datetime.fromtimestamp(x))
    df['dt_sunset']  = df['sunset'].apply(lambda x : datetime.fromtimestamp(x))
    df['dt_sunrise'] = pd.to_datetime(df["dt_sunrise"])
    df['dt_sunset']  = pd.to_datetime(df["dt_sunset"])
    df['lg_day']     = df["dt_sunset"] - df["dt_sunrise"]
    # conversion in hour
    df['lg_day']     = df["lg_day"].apply(lambda x : x.total_seconds()/60/60)

    return df

weather = create_features(all_forecast)
weather.head(5)

,dt,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,...,coord.lat,coord.lon,lat,lon,ville,date,dayofweek,dt_sunrise,dt_sunset,lg_day
0,1667304000,10000,0.39,2022-11-01 12:00:00,16.39,15.96,16.39,16.72,1015,1015,...,48.636,-1.5115,48.64,-1.51,Mont Saint Michel,2022-11-01 12:00:00+00:00,1,2022-11-01 07:52:02,2022-11-01 17:47:06,9.917778
1,1667314800,10000,1.00,2022-11-01 15:00:00,15.95,15.56,15.06,15.95,1015,1015,...,48.636,-1.5115,48.64,-1.51,Mont Saint Michel,2022-11-01 15:00:00+00:00,1,2022-11-01 07:52:02,2022-11-01 17:47:06,9.917778
2,1667325600,10000,1.00,2022-11-01 18:00:00,14.13,13.61,13.00,14.13,1016,1016,...,48.636,-1.5115,48.64,-1.51,Mont Saint Michel,2022-11-01 18:00:00+00:00,1,2022-11-01 07:52:02,2022-11-01 17:47:06,9.917778
3,1667336400,10000,0.07,2022-11-01 21:00:00,11.60,10.80,11.60,11.60,1019,1019,...,48.636,-1.5115,48.64,-1.51,Mont Saint Michel,2022-11-01 21:00:00+00:00,1,2022-11-01 07:52:02,2022-11-01 17:47:06,9.917778
4,1667347200,10000,0.15,2022-11-02 00:00:00,10.90,10.11,10.90,10.90,1020,1020,...,48.636,-1.5115,48.64,-1.51,Mont Saint Michel,2022-11-02 00:00:00+00:00,2,2022-11-01 07:52:02,2022-11-01 17:47:06,9.917778


## 3.2 feature selection

In [11]:
# mean weather on the 4 following days
feature_mean = ["main.temp", "main.feels_like", "main.temp_max", "main.humidity", "clouds.all", "wind.speed", "visibility", "lg_day"]
resumeW = weather.groupby(["ville", "lat", "lon"])[feature_mean].mean().reset_index()

# sum of rain
rain = weather.groupby(["ville", "lat", "lon"])["rain.3h"].sum().reset_index()
resumeW = resumeW.merge(rain, on = ["ville", "lat", "lon"])
resumeW.head()

,ville,lat,lon,main.temp,main.feels_like,main.temp_max,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h
0,Aigues Mortes,43.57,4.19,15.32625,14.55225,15.32625,62.725,45.250,5.50625,9935.700,10.276944,7.91
1,Aix en Provence,43.53,5.45,14.21025,13.08125,14.21025,62.525,42.700,4.31225,9870.925,10.279444,7.40
2,Amiens,49.89,2.30,11.34550,10.23125,11.35300,78.200,71.225,5.60150,9824.475,9.817778,25.43
3,Annecy,45.90,6.13,9.59200,9.07850,9.62500,86.675,78.200,1.25475,9423.300,10.120833,34.15
4,Ariege,48.41,2.79,11.52375,10.35600,11.53100,75.725,70.300,4.19600,9955.250,9.936111,9.36


In [12]:
# for categorical data: extraction of the most frequent category to describe the main weather
def freqfeature(df, df_to_merge, feature=None):
    df = df.copy()
    # calculation of the frequence of feature per city
    name   = "frequency_" + feature
    first  = df.groupby(["ville", "coord.lat", "coord.lon"])[feature].value_counts().to_frame(name=name).reset_index()

    # extraction of the most frequent feature
    list_column = [feature, first.columns[-1]]
    second = first.groupby(["ville", "coord.lat", "coord.lon"])[list_column].max().reset_index()

    # merge to the desired dataframe (dataframe with city ville, lat and lon columns)
    output = df_to_merge.merge(second.loc[:, ["ville", "coord.lat", "coord.lon", feature]])

    return(output)

In [13]:
feature_frequent = ["mainweather.main", "mainweather.description"]
for f in feature_frequent:
    final = freqfeature(df=weather, df_to_merge=resumeW, feature=f)
    resumeW = final

display(final.head())
print(final.shape)


,ville,lat,lon,main.temp,main.feels_like,main.temp_max,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,Aigues Mortes,43.57,4.19,15.32625,14.55225,15.32625,62.725,45.250,5.50625,9935.700,10.276944,7.91,43.5658,4.1913,Rain,scattered clouds
1,Aix en Provence,43.53,5.45,14.21025,13.08125,14.21025,62.525,42.700,4.31225,9870.925,10.279444,7.40,43.5298,5.4475,Rain,scattered clouds
2,Amiens,49.89,2.30,11.34550,10.23125,11.35300,78.200,71.225,5.60150,9824.475,9.817778,25.43,49.8942,2.2957,Rain,scattered clouds
3,Annecy,45.90,6.13,9.59200,9.07850,9.62500,86.675,78.200,1.25475,9423.300,10.120833,34.15,45.8992,6.1289,Rain,scattered clouds
4,Ariege,48.41,2.79,11.52375,10.35600,11.53100,75.725,70.300,4.19600,9955.250,9.936111,9.36,48.4103,2.7858,Rain,scattered clouds


(35, 16)


In [14]:
# criteria can be created in order to fit with different tastes of tourists
def df_criteria(df):
    # dataframe sortering values
    destination = {"warm" : df.sort_values(by = 'main.temp_max', ascending=False).head(5),
                   "landscape" : df.sort_values(by = 'visibility', ascending=False).head(5),
                   "less_cloud" : df.sort_values(by = 'clouds.all').head(5)}

    # dataframe which could be without data regarding the weather conditions
    ## sunny and warm
    try:
        1 / df.loc[df["mainweather.main"] == "Sun",:].shape[0]
    except ZeroDivisionError:
        print("There is no sunny days in France in the following days")
    else:
        sunny = df.loc[df["mainweather.main"] == "Sun",:]
        destination["sunny"] = sunny.sort_values(by = 'main.temp_like', ascending=False).head(5)

    ## snow
    try:
        1 / df.loc[df["mainweather.main"] == "Snow",:].shape[0]
    except ZeroDivisionError:
        print("There is no snow in France in the following days")
    else:
        snow = df.loc[df["mainweather.main"] == "Snow",:]        
        ### where this is higher amount of snow
        destination["snow"] = snow.sort_values(by = 'rain.3h', ascending=False).head(5)

    ## no_rain and low_wind
    try:
        1 / df.loc[df["mainweather.main"] != "Rain",:].shape[0]
    except ZeroDivisionError:
        print("There only rainy days in France in the following days")
    else:
        no_rain = df.loc[df["mainweather.main"] != "Rain",:]
        destination["no_rain_low_wind"] = no_rain.sort_values(by = 'wind.speed').head(5)
    
    return(destination)


In [15]:
destination = df_criteria(final)


There is no sunny days in France in the following days
There is no snow in France in the following days
There only rainy days in France in the following days


In [16]:
destination.keys()

dict_keys(['warm', 'landscape', 'less_cloud'])

# 4. Graph interactif top 5 destinations

## 4.1 preparation of the dictionnary

In [20]:
# needed information for plot per weather criteria
car_criteria = {"name" : ["warm", "landscape", "less_cloud", "sunny", "snow", "no_rain_low_wind"],
                "color_marker" : ["darkorange", "darkolivegreen", "royalblue", "gold", "lavender", "mediumpurple"],
                "init_visible" : [True, False, False, False, False, False],
                "label_button" : ["Hautes températures",
                                "Temps dégagé",
                                "Peu de nuages",
                                "Ensoleillé",
                                "Neige",
                                "Temps clair et calme"],
                "text_map" : [["ville", "main.feels_like", "main.temp_max", "main.humidity"],
                            ["ville", "visibility", "main.feels_like", "wind.speed"],
                            ["ville", "clouds.all", "main.feels_like", "rain.3h"],
                            ["ville", "main.feels_like", "main.temp_max", "lg_day"],
                            ["ville", "rain.3h", "clouds.all", "main.temp", "visibility"],
                            ["ville", "main.feels_like", "wind.speed", "clouds.all"]]
            }

car_criteria.keys()

dict_keys(['name', 'color_marker', 'init_visible', 'label_button', 'text_map'])

In [21]:
# automatic generation of the visibility argument in regard of the real criteria number
def auto_visibility_dict(dict_dest):
    list_criteria = list(dict_dest.keys())
    visible = [False for c in range(len(list(dict_dest.keys())))]

    list_visibility = list(range(len(list(dict_dest.keys()))))
    for c in range(len(list_criteria)):
        this_criteria = visible.copy()
        this_criteria[c] = True
        list_visibility[c] = this_criteria
    
    return(list_criteria, list_visibility)

In [22]:
list_criteria, list_visibility = auto_visibility_dict(destination)
print(list_criteria)
print(list_visibility)

['warm', 'landscape', 'less_cloud']
[[True, False, False], [False, True, False], [False, False, True]]


In [23]:
# comprehension dictionary compiling all the information and data per criteria
labels = [{'criteria' : list_criteria[c],
           'data': destination[list_criteria[c]],
           'title': car_criteria["label_button"][car_criteria["name"].index(list_criteria[c])],
           'color_marker': car_criteria["color_marker"][car_criteria["name"].index(list_criteria[c])],
           'init_visible': car_criteria["init_visible"][car_criteria["name"].index(list_criteria[c])],
           'text_map': car_criteria["text_map"][car_criteria["name"].index(list_criteria[c])],
           'visible' : list_visibility[c]
           } for c in range(len(list_criteria))]

len(labels)

3

In [24]:
labels

[{'criteria': 'warm',
  'data':                        ville    lat   lon  main.temp  main.feels_like  \
  12                    Cassis  43.21  5.54   16.71725         15.96550   
  24                 Marseille  43.30  5.37   16.69775         16.00075   
  14                 Collioure  42.53  3.08   16.33550         15.63400   
  30  Saintes Maries de la mer  43.45  4.43   15.77575         15.08900   
  10        Bormes les Mimosas  43.15  6.34   15.40800         14.69400   
  
      main.temp_max  main.humidity  clouds.all  wind.speed  visibility  \
  12       16.73150         58.250      39.250     6.54925    9875.075   
  24       16.70950         60.425      37.800     7.12625   10000.000   
  14       16.33550         61.650      54.050     6.07475   10000.000   
  30       15.77575         64.325      47.975     7.38200   10000.000   
  10       15.50125         64.700      35.250     4.73325   10000.000   
  
         lg_day  rain.3h  coord.lat  coord.lon mainweather.main  \
  1

## 4.2 Graph

In [111]:
text_cat = {"ville":"ville",
            "main.feels_like":"température ressentie (°C)", "main.temp_max":"température maximale (°C)", "main.humidity":"humidité",
            "visibility":"visibilité (m)", "wind.speed":"vitesse vent (m/sec)", "clouds.all":"couverture nuageuse (%)",
            "rain.3h":"pluie (mm)", "lg_day":"longueur du jour", "main.temp":"température moyenne (°C)"}

In [112]:
fig = go.Figure()

for l in range(len(labels)):
    thisC = labels[l]
    # text on the plot
    text_data = thisC['data'].loc[:,thisC["text_map"]]

    fig.add_trace(
    go.Scattermapbox(
        lat= thisC['data']['coord.lat'],
        lon= thisC['data']["coord.lon"],
        visible=thisC['init_visible'],
        mode='markers+text',
        marker=go.scattermapbox.Marker(size=14, color=thisC["color_marker"]),
        
        customdata=[[text_data.iloc[r,c] for c in range(len(thisC["text_map"]))] for r in range(text_data.shape[0])],
        hovertemplate="<b>Ville: </b> %{customdata[0]} <br>" + text_cat[text_data.columns[1]] + ": </b> %{customdata[1]} <br>" + text_cat[text_data.columns[2]] + ":</b> %{customdata[2]} <br>" + text_cat[text_data.columns[3]] + ":</b> %{customdata[3]} <br>"
    ))
    fig.update_traces(hovertemplate=None, selector={'name':labels[l]['criteria']})

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom = 4.5,
    mapbox_center=go.layout.mapbox.Center(lat=47.8, lon=2.3),
    title = go.layout.Title(text = "Top 5 destinations in France", x = 0.5),
    showlegend = False,
    width=850,
    height=800)

fig.update_layout(
    updatemenus = [go.layout.Updatemenu(
        active = 0,
        buttons = [go.layout.updatemenu.Button(
                        label = labels[l]["title"],
                        method = "update",
                        args = [{"visible" : labels[l]["visible"]}])
                        for l in range(len(labels))]
    )]
)

fig.show(renderer="vscode")

## 4.3 Savings

In [298]:
# save forecast for all city
final.to_csv('src/forecast_final.csv')

# save forecast for city to scrap
dest_dict = [{"criteria":d,
              "data": destination[d].to_dict(orient='list')
              } for d in list(destination.keys())]

import json
with open('src/forecast_destination.json', 'w') as f:
    json.dump(dest_dict, f)

In [16]:
# Saving list of city to scrap without duplicates

# using numpy.unique
import numpy as np
  
# function to get unique values  
def unique(list1):
    x = set(list1)
    return(list(x))

list_ville = []
for d in list(destination.keys()):
    for v in range(len(destination[d]["ville"])):
        list_ville.append(destination[d]["ville"].values[v])

print("length of the list:", len(list_ville), "\n")
print("length of the unique values:", len(unique(list_ville)))
print("list of city to scrap:", unique(list_ville))



length of the list: 15 

length of the unique values: 11
list of city to scrap: ['Montauban', 'Strasbourg', 'Collioure', 'Avignon', 'Gorges du Verdon', 'Aix en Provence', 'Marseille', 'Bormes les Mimosas', 'Nimes', 'Saintes Maries de la mer', 'Cassis']


In [25]:
# save this csv for scraping
np.savetxt('src/city_destination.csv', list_ville, delimiter =", ", fmt ='% s')

# 5. Booking scraping

done the 5th and 6th of November, 2022

## 5.1 spider hotel

- Treatment of the spider 'hotel.py' results

In [59]:
# upload first file from first spider 'hotel.py'
#the_file = open("src/list_hotel.json")
the_file = open("src/url_hotel.json")
hotel = json.load(the_file)
the_file.close()

In [60]:
hotel_list = [pd.DataFrame(hotel[x]) for x in range(len(hotel))]
len(hotel_list)

75

In [61]:
# concatenation of dataframes
hotel_key = list(hotel[0].keys()) + ["city", "order"]
hotel_df = pd.DataFrame(columns=hotel_key)
for x in range(len(hotel_list)):
    this_hotel = hotel_list[x]
    this_hotel["city"] = this_hotel["url_tested"][0].split("=")[1].split("&")[0].replace("%20", " ")
    this_hotel["order"] = this_hotel["url_tested"][0].split("=")[2]
    hotel_df = pd.concat([hotel_df, this_hotel])

print(hotel_df.shape)
display(hotel_df.head())

(1875, 5)


,url_tested,name_hotel,url_hotel,city,order
0,https://www.booking.com/searchresults.fr.html?...,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER,https://www.booking.com/hotel/fr/une-terrasse-...,Cassis,popularity
1,https://www.booking.com/searchresults.fr.html?...,Villa ANTINEA,https://www.booking.com/hotel/fr/antinea-cassi...,Cassis,popularity
2,https://www.booking.com/searchresults.fr.html?...,LE REVE BLEU,https://www.booking.com/hotel/fr/le-reve-bleu-...,Cassis,popularity
3,https://www.booking.com/searchresults.fr.html?...,Le chemin de belle fille,https://www.booking.com/hotel/fr/le-chemin-de-...,Cassis,popularity
4,https://www.booking.com/searchresults.fr.html?...,Home Cassis - Maison Julou - Piscine chauffée,https://www.booking.com/hotel/fr/home-cassis-m...,Cassis,popularity


In [62]:
hotel_df["url_short"] = hotel_df["url_hotel"].apply(lambda x : x.split("?")[0])
hotel_df.head(2)

,url_tested,name_hotel,url_hotel,city,order,url_short
0,https://www.booking.com/searchresults.fr.html?...,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER,https://www.booking.com/hotel/fr/une-terrasse-...,Cassis,popularity,https://www.booking.com/hotel/fr/une-terrasse-...
1,https://www.booking.com/searchresults.fr.html?...,Villa ANTINEA,https://www.booking.com/hotel/fr/antinea-cassi...,Cassis,popularity,https://www.booking.com/hotel/fr/antinea-cassi...


In [64]:
print(hotel_df.iloc[1,0])
print(hotel_df.iloc[1,2])
print(hotel_df.iloc[1,5])

https://www.booking.com/searchresults.fr.html?ss=Cassis&order=popularity
https://www.booking.com/hotel/fr/antinea-cassis12.fr.html?aid=304142&label=gen173nr-1FCAQoggJCDXNlYXJjaF9jYXNzaXNIDVgEaBWIAQGYAQ24ARjIAQzYAQHoAQH4AQOIAgGoAgS4AovQmZsGwAIB0gIkNmQzYTJhN2MtYjU5OC00YzkwLTg3M2ItMGEzY2NmODBmMjk32AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=361b60450e6701e7&srepoch=1667655692&from_beach_sr=1&beach_sr_walking_distance=874&from=searchresults#hotelTmpl
https://www.booking.com/hotel/fr/antinea-cassis12.fr.html


In [8]:
# save this csv for scraping
hotel_df.to_csv('src/list_hotel_clean.csv')

## 5.2 spider car_hotel

In [111]:
# upload first file from first spider 'car_hotel.py'
the_file = open("src/car_hotel.json")
hotel = json.load(the_file)
the_file.close()

In [66]:
hotel[0].keys()

dict_keys(['url_hotel', 'description_hotel', 'score_hotel', 'coordinates_hotel', 'adresse_hotel', 'animals_hotel'])

In [67]:
len(hotel)

1871

In [68]:
hotel[0]

{'url_hotel': 'https://www.booking.com/hotel/fr/une-terrasse-sur-la-mer-cassis.fr.html',
 'description_hotel': ["Vous pouvez bénéficier d'une réduction Genius dans l'établissement UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER\xa0! ",
  ' pour économiser.',
  "Situé à Cassis, à seulement 400 mètres de Bestouan, l'établissement UNE TERRASSE SUR LA MER 3 Chambre 2 Personnes L'EVI MER propose un hébergement en bord de mer avec une terrasse et une connexion Wi-Fi gratuite. Il est situé à 1,1 km de la Grande Mer et dispose d'un bureau d'excursions.",
  "Cet appartement comprend 2 chambres, une salle de bains, du linge de lit, des serviettes, une télévision à écran plat, un coin repas, une cuisine entièrement équipée et un balcon avec vue sur la mer. Pour plus de commodité, l'établissement peut fournir des serviettes et du linge de lit moyennant des frais supplémentaires.",
  'Un service de location de vélos est assuré sur place pour vous permettre de partir en randonnée dans les environs

In [112]:
# results concatenation
total = pd.DataFrame()
for h in range(len(hotel)):
    thisH = hotel[h].copy()
    thisH['description_hotel'] = "".join(thisH['description_hotel'])
    thisH["hotel_index"] = h
    thisH = pd.DataFrame([thisH])

    total = pd.concat([total, thisH])

total.head(2)

,url_hotel,description_hotel,score_hotel,coordinates_hotel,adresse_hotel,animals_hotel,hotel_index
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","43.21407197,5.53062182","\n9 Avenue Maurice Jermini, 13260 Cassis, Fran...",Cet hébergement est non-fumeurs.,0
0,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","43.22024010,5.54246930","\n2 Avenue Auguste Favier, 13260 Cassis, France\n",Cet hébergement est non-fumeurs.,1


In [113]:
# formating the final dataframe describing hotels
total['lat'] = total['coordinates_hotel'].apply(lambda x : x.split(",")[0])
total['lon'] = total['coordinates_hotel'].apply(lambda x : x.split(",")[1])

total['description_hotel'] = total['description_hotel'].apply(lambda x : x.replace("\xa0", ""))
total['description_hotel'] = total['description_hotel'].apply(lambda x : x.replace("\n", ""))
total['adresse_hotel']     = total['adresse_hotel'].apply(lambda x : x.replace("\n", ""))
total['animals_hotel']     = total['animals_hotel'].apply(lambda x : str(x).replace("\n", ""))

In [114]:
total.shape

(1871, 9)

## 5.3 merging all the information

In [110]:
# uploading the initial list of hotel to scrap
hotel_to_scrap = pd.read_csv('src/list_hotel_clean.csv')
hotel_to_scrap.head(2)

,Unnamed: 0,url_tested,name_hotel,url_hotel,city,order,url_short
0,0,https://www.booking.com/searchresults.fr.html?...,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER,https://www.booking.com/hotel/fr/une-terrasse-...,Cassis,popularity,https://www.booking.com/hotel/fr/une-terrasse-...
1,1,https://www.booking.com/searchresults.fr.html?...,Villa ANTINEA,https://www.booking.com/hotel/fr/antinea-cassi...,Cassis,popularity,https://www.booking.com/hotel/fr/antinea-cassi...


In [146]:
def format_hotel(data, data_car):
    # purpose : to have a columns per order criteria and 1 if the hotel was appearing in the list of hotel to scrap regarding this criteria
    # data : list of hotel to scrap, order and description -----> here, dataframe hotel (from list_hotel_clean.csv)
    # data_car : caracteristic of hotel ----> here, dataframe total

    # preparation
    list_criteria = data["order"].unique()
    tested_url = data_car["url_hotel"].unique()
    data_car["city"] = "to_fill"
    data_car["name_hotel"] = "to_fill"

    # adding order criteria
    for d in list_criteria:
        # adding colums to fill to data
        data_car[d] = 0
        list_hotel = list(data.loc[data["order"] == d, "url_short"])
        
        # fill with 1 if the hotel is selected by the criteria
        data_car.loc[data_car["url_hotel"].isin(list_hotel), d] = 1

    # adding name of the city
    for c in tested_url:
        data_car.loc[data_car["url_hotel"] == c, "city"] = data.loc[data["url_short"] == c, "city"].unique()[0]
        data_car.loc[data_car["url_hotel"] == c, "name_hotel"] = data.loc[data["url_short"] == c, "name_hotel"].unique()[0]

    return(data_car)

In [147]:
# adding order selction from the initial list of hotel to scrap to the total dataframe (i.e. caracteristics of all hotels)
those_hotel = format_hotel(hotel_to_scrap, total)
# removing coordinates_hotel, url_tested, hotel_index
those_hotel = those_hotel.drop(["coordinates_hotel", "hotel_index"], axis = 1)

display(those_hotel.head())
those_hotel.shape

,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,upsort_bh,class,distance_from_search,bayesian_review_score,name_hotel
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.21407197,5.53062182,Cassis,1,1,1,0,0,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER
0,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.22024010,5.54246930,Cassis,1,1,0,0,0,Villa ANTINEA
0,https://www.booking.com/hotel/fr/le-reve-bleu-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,6","28 Avenue Jules Ferry, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.22062430,5.54076700,Cassis,1,1,1,0,0,LE REVE BLEU
0,https://www.booking.com/hotel/fr/le-chemin-de-...,"Situé à Cassis, l'établissement Le chemin de b...","7,4","3 Route Pierre Imbert, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.21641033,5.55070017,Cassis,1,0,0,0,0,Le chemin de belle fille
0,https://www.booking.com/hotel/fr/home-cassis-m...,"Offrant une vue sur le jardin, le Home Cassis ...","7,5","23 Avenue de la Marne, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.22207969,5.53792695,Cassis,1,1,0,0,0,Home Cassis - Maison Julou - Piscine chauffée


(1871, 14)

In [148]:
# saving
those_hotel.to_csv('src/hotel_description_order.csv')

# 6. Importation to datalake and database

##### Steps:
- to store in a datalake: creation bucket on AWS S3 service
- to create a database (data warehouse): AWS RDS service


##### Objects to store in the datalake:
- src/forecast_final.csv
- src/hotel_description_order.csv
- src/conplete_dataset.csv -> to create

##### In the database:
- src/complete_dataset_S3.csv

## 6.1 preparing working dataset

- ddl destination

In [150]:
with open('src/forecast_destination.json') as f:
    destination = f.read()

In [151]:
destination = json.loads(destination)

In [152]:
list_destination = {destination[d]["criteria"]:destination[d]["data"]["ville"] for d in range(len(destination))}
list_destination

{'warm': ['Cassis',
  'Marseille',
  'Collioure',
  'Saintes Maries de la mer',
  'Bormes les Mimosas'],
 'landscape': ['Gorges du Verdon',
  'Nimes',
  'Strasbourg',
  'Montauban',
  'Avignon'],
 'less_cloud': ['Bormes les Mimosas',
  'Marseille',
  'Cassis',
  'Aix en Provence',
  'Nimes']}

- ddl total

In [153]:
total = pd.read_csv("src/hotel_description_order.csv")
total = total.drop("Unnamed: 0", axis=1)
total.head(2)

,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,upsort_bh,class,distance_from_search,bayesian_review_score,name_hotel
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,1,1,0,0,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER
1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,1,0,0,0,Villa ANTINEA


- function and formatting

In [154]:
def format_final(data_hotel, data_destination):
    # purpose : to have a columns per criteria and 1 if the city was selected by this criteria
    # data_hotel : list of hotel, order and description -----> here, dataframe those_hotel (from hotel_description_order.csv)
    # data_destination : from API weather requests, list of city regarding weather criteria -----> here, dictionary destination (formated in list_destination, from forecast_destination.json)
    
    for d in data_destination.keys():
        # adding colums to fill to data_hotel
        data_hotel[d] = 0
        # fill with 1 if the city is selected by the criteria
        data_hotel.loc[data_hotel["city"].isin(data_destination[d]), d] = 1       
    
    return(data_hotel)

In [155]:
dataset = format_final(data_hotel=total, data_destination=list_destination)
dataset.head()

,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,upsort_bh,class,distance_from_search,bayesian_review_score,name_hotel,warm,landscape,less_cloud
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,1,1,0,0,UNE TERRASSE SUR LA MER 3 pers 2 chambres VUE MER,1,0,1
1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,1,0,0,0,Villa ANTINEA,1,0,1
2,https://www.booking.com/hotel/fr/le-reve-bleu-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,6","28 Avenue Jules Ferry, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220624,5.540767,Cassis,1,1,1,0,0,LE REVE BLEU,1,0,1
3,https://www.booking.com/hotel/fr/le-chemin-de-...,"Situé à Cassis, l'établissement Le chemin de b...","7,4","3 Route Pierre Imbert, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.216410,5.550700,Cassis,1,0,0,0,0,Le chemin de belle fille,1,0,1
4,https://www.booking.com/hotel/fr/home-cassis-m...,"Offrant une vue sur le jardin, le Home Cassis ...","7,5","23 Avenue de la Marne, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.222080,5.537927,Cassis,1,1,0,0,0,Home Cassis - Maison Julou - Piscine chauffée,1,0,1


In [156]:
dataset.shape

(1871, 17)

- adding forecast to the complete dataset

In [157]:
forecast = pd.read_csv('src/forecast_final.csv')
forecast = forecast.drop("Unnamed: 0", axis=1)
forecast.head()

,ville,lat,lon,main.temp,main.feels_like,main.temp_max,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,Aigues Mortes,43.57,4.19,15.32625,14.55225,15.32625,62.725,45.250,5.50625,9935.700,10.276944,7.91,43.5658,4.1913,Rain,scattered clouds
1,Aix en Provence,43.53,5.45,14.21025,13.08125,14.21025,62.525,42.700,4.31225,9870.925,10.279444,7.40,43.5298,5.4475,Rain,scattered clouds
2,Amiens,49.89,2.30,11.34550,10.23125,11.35300,78.200,71.225,5.60150,9824.475,9.817778,25.43,49.8942,2.2957,Rain,scattered clouds
3,Annecy,45.90,6.13,9.59200,9.07850,9.62500,86.675,78.200,1.25475,9423.300,10.120833,34.15,45.8992,6.1289,Rain,scattered clouds
4,Ariege,48.41,2.79,11.52375,10.35600,11.53100,75.725,70.300,4.19600,9955.250,9.936111,9.36,48.4103,2.7858,Rain,scattered clouds


In [158]:
forecast = forecast.rename(columns={"ville":"city"})
forecast = forecast.drop("lat", axis=1)
forecast = forecast.drop("lon", axis=1)
forecast.head(2)

,city,main.temp,main.feels_like,main.temp_max,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,Aigues Mortes,15.32625,14.55225,15.32625,62.725,45.25,5.50625,9935.700,10.276944,7.91,43.5658,4.1913,Rain,scattered clouds
1,Aix en Provence,14.21025,13.08125,14.21025,62.525,42.70,4.31225,9870.925,10.279444,7.40,43.5298,5.4475,Rain,scattered clouds


In [159]:
working = dataset.merge(forecast, on="city", how="left")
print(working.shape)
display(working.head())

(1871, 30)


,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,upsort_bh,...,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
2,https://www.booking.com/hotel/fr/le-reve-bleu-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,6","28 Avenue Jules Ferry, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220624,5.540767,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
3,https://www.booking.com/hotel/fr/le-chemin-de-...,"Situé à Cassis, l'établissement Le chemin de b...","7,4","3 Route Pierre Imbert, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.216410,5.550700,Cassis,1,0,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
4,https://www.booking.com/hotel/fr/home-cassis-m...,"Offrant une vue sur le jardin, le Home Cassis ...","7,5","23 Avenue de la Marne, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.222080,5.537927,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds


In [160]:
# saving
working.to_csv('src/complete_dataset.csv')

## 6.2 Export to S3

- Instructions:
Create your data lake using S3
Once you managed to build your dataset, you should store into S3 as a csv file.

In [175]:
import boto3

aws_credent = pd.read_csv('aws_bucket_credentials.csv', sep=';')

session = boto3.Session(aws_access_key_id=aws_credent["aws_access_key_id"].values[0], 
                        aws_secret_access_key=aws_credent["aws_secret_access_key"].values[0])

s3 = session.resource("s3")

In [176]:
# UpLoading raw datasets
## forecast dataset
s3.meta.client.upload_file(Filename="src/forecast_final.csv", Bucket="jedha092022", Key="forecast.csv")

## hotel description dataset
s3.meta.client.upload_file(Filename="src/hotel_description_order.csv", Bucket="jedha092022", Key="hotel.csv")

In [177]:
# UpLoading working dataframe
s3.meta.client.upload_file(Filename="src/complete_dataset.csv", Bucket="jedha092022", Key="complete.csv")

## 6.3 Export as database

- Instructions: Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB.

In [178]:
import sqlalchemy

In [184]:
# sql alchemy to create database
from sqlalchemy import create_engine, text

credent = pd.read_csv('pgadmin_credentials.csv', sep=';')

DBHOST = credent["DBHOST"].values[0]
DBUSER = credent["DBUSER"].values[0]
DBPASS = credent["DBPASS"].values[0]
DBNAME = credent["DBNAME"].values[0]
PORT   = credent["PORT"].values[0]

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

### 6.3.1 Import from S3 "complete_dataset"

In [185]:
s3 = session.resource('s3')
s3.meta.client.download_file(Bucket="jedha092022", Key="complete.csv", Filename="src/complete_dataset_S3.csv")

In [181]:
working = pd.read_csv("src/complete_dataset_S3.csv")
print(working.shape)
display(working.head(2))

(1871, 31)


,Unnamed: 0,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,...,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
1,1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds


In [182]:
working = working.drop("Unnamed: 0", axis=1)
working.columns

Index(['url_hotel', 'description_hotel', 'score_hotel', 'adresse_hotel',
       'animals_hotel', 'lat', 'lon', 'city', 'popularity', 'upsort_bh',
       'class', 'distance_from_search', 'bayesian_review_score', 'name_hotel',
       'warm', 'landscape', 'less_cloud', 'main.temp', 'main.feels_like',
       'main.temp_max', 'main.humidity', 'clouds.all', 'wind.speed',
       'visibility', 'lg_day', 'rain.3h', 'coord.lat', 'coord.lon',
       'mainweather.main', 'mainweather.description'],
      dtype='object')

### 6.3.2 Export to RDS database

In [186]:
working.to_sql('complete_dataset', engine, if_exists="replace", index=False)

2023-01-06 18:09:22,635 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-01-06 18:09:22,637 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-06 18:09:22,706 INFO sqlalchemy.engine.Engine select current_schema()
2023-01-06 18:09:22,707 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-06 18:09:22,772 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-01-06 18:09:22,773 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-06 18:09:22,839 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-01-06 18:09:22,840 INFO sqlalchemy.engine.Engine [generated in 0.00192s] {'name': 'complete_dataset'}
2023-01-06 18:09:22,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-06 18:09:22,939 INFO sqlalchemy.engine.Engine 
CREATE TABLE complete_dataset (
	url_hotel TEXT, 
	description_hotel TEXT, 
	score_hotel TEXT, 
	adresse_hotel TEXT, 
	animal

871

### 6.3.3 SQL request to test import success

In [187]:
stmt = text("SELECT * FROM complete_dataset")

# Create a dataFrame using SQL
quer = pd.read_sql(
        stmt,
        engine
    )

quer.head()

2023-01-06 18:09:37,596 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-01-06 18:09:37,598 INFO sqlalchemy.engine.Engine [cached since 14.76s ago] {'name': 'SELECT * FROM complete_dataset'}
2023-01-06 18:09:37,712 INFO sqlalchemy.engine.Engine SELECT * FROM complete_dataset
2023-01-06 18:09:37,713 INFO sqlalchemy.engine.Engine [generated in 0.00150s] {}


,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,upsort_bh,...,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
2,https://www.booking.com/hotel/fr/le-reve-bleu-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,6","28 Avenue Jules Ferry, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220624,5.540767,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
3,https://www.booking.com/hotel/fr/le-chemin-de-...,"Situé à Cassis, l'établissement Le chemin de b...","7,4","3 Route Pierre Imbert, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.216410,5.550700,Cassis,1,0,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
4,https://www.booking.com/hotel/fr/home-cassis-m...,"Offrant une vue sur le jardin, le Home Cassis ...","7,5","23 Avenue de la Marne, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.222080,5.537927,Cassis,1,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds


# 7. Interactive map of hotel

In [116]:
working = pd.read_csv("src/complete_dataset_S3.csv")
print(working.shape)
display(working.head(2))

(1871, 31)


,Unnamed: 0,url_hotel,description_hotel,score_hotel,adresse_hotel,animals_hotel,lat,lon,city,popularity,...,main.humidity,clouds.all,wind.speed,visibility,lg_day,rain.3h,coord.lat,coord.lon,mainweather.main,mainweather.description
0,0,https://www.booking.com/hotel/fr/une-terrasse-...,Vous pouvez bénéficier d'une réduction Genius ...,"8,5","9 Avenue Maurice Jermini, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.214072,5.530622,Cassis,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds
1,1,https://www.booking.com/hotel/fr/antinea-cassi...,Vous pouvez bénéficier d'une réduction Genius ...,"7,0","2 Avenue Auguste Favier, 13260 Cassis, France",Cet hébergement est non-fumeurs.,43.220240,5.542469,Cassis,1,...,58.25,39.25,6.54925,9875.075,10.299444,9.81,43.214,5.5396,Rain,scattered clouds


In [117]:
working = working.drop(columns='Unnamed: 0', axis=1)
working.columns

Index(['url_hotel', 'description_hotel', 'score_hotel', 'adresse_hotel',
       'animals_hotel', 'lat', 'lon', 'city', 'popularity', 'upsort_bh',
       'class', 'distance_from_search', 'bayesian_review_score', 'name_hotel',
       'warm', 'landscape', 'less_cloud', 'main.temp', 'main.feels_like',
       'main.temp_max', 'main.humidity', 'clouds.all', 'wind.speed',
       'visibility', 'lg_day', 'rain.3h', 'coord.lat', 'coord.lon',
       'mainweather.main', 'mainweather.description'],
      dtype='object')

## 7.1 plot preparation

- function

In [118]:
def plotResults(dataset, weather="warm", order_hotel="distance_from_search"):
    list_labels = ['popularity', 'upsort_bh', 'class', 'distance_from_search', 'bayesian_review_score']
    list_weather = ['warm', 'landscape', 'less_cloud', 'snow', 'sunny', 'no_rain_low_wind']
    
    # test of order_values
    if not order_hotel in list_labels:
        raise TypeError("Only those values are allowed: 'popularity', 'upsort_bh', 'class', 'distance_from_search', 'bayesian_review_score'")
    
    # test weather
    if not any(w in working.columns for w in list_weather):
        raise TypeError("Only those values are allowed: 'warm', 'landscape', 'less_cloud', 'snow', 'sunny', 'no_rain_low_wind'\nif you tried with correct value in lowercase letters, there is not columns in the dataframe matching with it.")

    # plot title
    order_type={'popularity':'Booking favorites', 'upsort_bh':'avalaible full accomodation', 'class':'star', 'distance_from_search':'downtown distance', 'bayesian_review_score':'best score'}
    weather_type={'warm':'a warm', 'landscape':'a high landscape view', 'less_cloud':'a not cloudy', 'snow':'a snowy', 'sunny':'a sunny', 'no_rain_low_wind':'a not rainy and windy'}
    title_plot=("Your next vacation in France for " + weather_type[weather] + " weather among those hotels ordered by " + order_type[order_hotel])

    # weather characteristics

    # data extraction
    data_to_plot = dataset.loc[(dataset[weather] == 1) & (dataset[order_hotel] == 1),:]
    
    # figure
    fig = px.scatter_mapbox(data_to_plot, lon = "lon", lat = "lat",
                            color = "city", labels={"city":"City to go"},
                            hover_data = ['city', 'name_hotel', 'score_hotel', 'adresse_hotel'],
                            color_continuous_scale='Portland', height=800, width=1000, zoom=4.5)
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":50,"t":100,"l":50,"b":10})
    fig.update_layout(title=title_plot)
    
    return(fig.show())


## 7.2 Plot

In [190]:
list_weather = ['warm', 'landscape', 'less_cloud', 'snow', 'sunny', 'no_rain_low_wind']

for w in list_weather:
    if w in working.columns :
        print(w)

warm
landscape
less_cloud


In [191]:
plotResults(dataset=working, weather="landscape", order_hotel="best")

TypeError: Only those values are allowed: 'popularity', 'upsort_bh', 'class', 'distance_from_search', 'bayesian_review_score'

In [119]:
plotResults(dataset=working, weather="landscape", order_hotel="bayesian_review_score")

In [120]:
plotResults(dataset=working, weather="less_cloud", order_hotel="upsort_bh")